# TV Script Generation

In this project, you'll generate your own [Seinfeld](https://en.wikipedia.org/wiki/Seinfeld) TV scripts using RNNs.  You'll be using part of the [Seinfeld dataset](https://www.kaggle.com/thec03u5/seinfeld-chronicles#scripts.csv) of scripts from 9 seasons.  The Neural Network you'll build will generate a new ,"fake" TV script, based on patterns it recognizes in this training data.

## Get the Data

The data is already provided for you in `./data/Seinfeld_Scripts.txt` and you're encouraged to open that file and look at the text. 
>* As a first step, we'll load in this data and look at some samples. 
* Then, you'll be tasked with defining and training an RNN to generate a new script!

In [2]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# load in data
import helper
data_dir = './data/Seinfeld_Scripts.txt'
text = helper.load_data(data_dir)

## Explore the Data
Play around with `view_line_range` to view different parts of the data. This will give you a sense of the data you'll be working with. You can see, for example, that it is all lowercase text, and each new line of dialogue is separated by a newline character `\n`.

In [ ]:
view_line_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))

lines = text.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))

print()
print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(text.split('\n')[view_line_range[0]:view_line_range[1]]))

---
## Implement Pre-processing Functions
The first thing to do to any dataset is pre-processing.  Implement the following pre-processing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following **tuple** `(vocab_to_int, int_to_vocab)`

In [3]:
import problem_unittests as tests
from collections import Counter


def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    
    counts = Counter(text)
    vocab = sorted(counts, key=counts.get, reverse=True)
    vocab_to_int = {word: ii for ii, word in enumerate(vocab, 0)}
    
    int_to_vocab = dict((v,k) for k,v in vocab_to_int.items())
    # return tuple
    return (vocab_to_int, int_to_vocab)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks can create multiple ids for the same word. For example, "bye" and "bye!" would generate two different word ids.

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( **.** )
- Comma ( **,** )
- Quotation Mark ( **"** )
- Semicolon ( **;** )
- Exclamation mark ( **!** )
- Question mark ( **?** )
- Left Parentheses ( **(** )
- Right Parentheses ( **)** )
- Dash ( **-** )
- Return ( **\n** )

This dictionary will be used to tokenize the symbols and add the delimiter (space) around it.  This separates each symbols as its own word, making it easier for the neural network to predict the next word. Make sure you don't use a value that could be confused as a word; for example, instead of using the value "dash", try using something like "||dash||".

In [ ]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenized dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    mydict = {'.':'||Period||',',':'||Comma||','"':'||Quotation||',';':'||Semicolon||','!':'||Exlamation||','?':'||Question||','(':'||LeftParant||',')':'||RightParant||','-':'||Dash||','\n':'||Return||'}
    return mydict

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

## Pre-process all the data and save it

Running the code cell below will pre-process all the data and save it to file. You're encouraged to lok at the code for `preprocess_and_save_data` in the `helpers.py` file to see what it's doing in detail, but you do not need to change this code.

In [ ]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# pre-process training data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [26]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
In this section, you'll build the components necessary to build an RNN by implementing the RNN Module and forward and backpropagation functions.

### Check Access to GPU

In [27]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch

# Check for a GPU
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('No GPU found. Please use a GPU to train your neural network.')

## Input
Let's start with the preprocessed input data. We'll use [TensorDataset](http://pytorch.org/docs/master/data.html#torch.utils.data.TensorDataset) to provide a known format to our dataset; in combination with [DataLoader](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader), it will handle batching, shuffling, and other dataset iteration functions.

You can create data with TensorDataset by passing in feature and target tensors. Then create a DataLoader as usual.
```
data = TensorDataset(feature_tensors, target_tensors)
data_loader = torch.utils.data.DataLoader(data, 
                                          batch_size=batch_size)
```

### Batching
Implement the `batch_data` function to batch `words` data into chunks of size `batch_size` using the `TensorDataset` and `DataLoader` classes.

>You can batch words using the DataLoader, but it will be up to you to create `feature_tensors` and `target_tensors` of the correct size and content for a given `sequence_length`.

For example, say we have these as input:
```
words = [1, 2, 3, 4, 5, 6, 7]
sequence_length = 4
```

Your first `feature_tensor` should contain the values:
```
[1, 2, 3, 4]
```
And the corresponding `target_tensor` should just be the next "word"/tokenized word value:
```
5
```
This should continue with the second `feature_tensor`, `target_tensor` being:
```
[2, 3, 4, 5]  # features
6             # target
```

In [28]:
from torch.utils.data import TensorDataset, DataLoader


def batch_data(words, sequence_length, batch_size):
    """
    Batch the neural network data using DataLoader
    :param words: The word ids of the TV scripts
    :param sequence_length: The sequence length of each batch
    :param batch_size: The size of each batch; the number of sequences in a batch
    :return: DataLoader with batched data
    """
    # TODO: Implement function
    feature = []
    target = []
    for i in range(len(words)//sequence_length):
        feature.append(words[i:(i+sequence_length)])
        target.append(words[i+sequence_length])
    feature_tensors = torch.from_numpy(np.asarray(feature))
    target_tensors = torch.from_numpy(np.asarray(target))
    data = TensorDataset(feature_tensors, target_tensors)
    dataloader = DataLoader(data,batch_size=batch_size)
    # return a dataloader
    return dataloader

# there is no test for this function, but you are encouraged to create
# print statements and tests of your own


### Test your dataloader 

You'll have to modify this code to test a batching function, but it should look fairly similar.

Below, we're generating some test text data and defining a dataloader using the function you defined, above. Then, we are getting some sample batch of inputs `sample_x` and targets `sample_y` from our dataloader.

Your code should return something like the following (likely in a different order, if you shuffled your data):

```
torch.Size([10, 5])
tensor([[ 28,  29,  30,  31,  32],
        [ 21,  22,  23,  24,  25],
        [ 17,  18,  19,  20,  21],
        [ 34,  35,  36,  37,  38],
        [ 11,  12,  13,  14,  15],
        [ 23,  24,  25,  26,  27],
        [  6,   7,   8,   9,  10],
        [ 38,  39,  40,  41,  42],
        [ 25,  26,  27,  28,  29],
        [  7,   8,   9,  10,  11]])

torch.Size([10])
tensor([ 33,  26,  22,  39,  16,  28,  11,  43,  30,  12])
```

### Sizes
Your sample_x should be of size `(batch_size, sequence_length)` or (10, 5) in this case and sample_y should just have one dimension: batch_size (10). 

### Values

You should also notice that the targets, sample_y, are the *next* value in the ordered test_text data. So, for an input sequence `[ 28,  29,  30,  31,  32]` that ends with the value `32`, the corresponding output should be `33`.

In [29]:
# test dataloader
import numpy as np
test_text = range(50)
t_loader = batch_data(test_text, sequence_length=5, batch_size=10)

data_iter = iter(t_loader)
sample_x, sample_y = data_iter.next()

print(sample_x.shape)
print(sample_x)
print()
print(sample_y.shape)
print(sample_y)

torch.Size([10, 5])
tensor([[  0,   1,   2,   3,   4],
        [  1,   2,   3,   4,   5],
        [  2,   3,   4,   5,   6],
        [  3,   4,   5,   6,   7],
        [  4,   5,   6,   7,   8],
        [  5,   6,   7,   8,   9],
        [  6,   7,   8,   9,  10],
        [  7,   8,   9,  10,  11],
        [  8,   9,  10,  11,  12],
        [  9,  10,  11,  12,  13]])

torch.Size([10])
tensor([  5,   6,   7,   8,   9,  10,  11,  12,  13,  14])


---
## Build the Neural Network
Implement an RNN using PyTorch's [Module class](http://pytorch.org/docs/master/nn.html#torch.nn.Module). You may choose to use a GRU or an LSTM. To complete the RNN, you'll have to implement the following functions for the class:
 - `__init__` - The initialize function. 
 - `init_hidden` - The initialization function for an LSTM/GRU hidden state
 - `forward` - Forward propagation function.
 
The initialize function should create the layers of the neural network and save them to the class. The forward propagation function will use these layers to run forward propagation and generate an output and a hidden state.

**The output of this model should be the *last* batch of word scores** after a complete sequence has been processed. That is, for each input sequence of words, we only want to output the word scores for a single, most likely, next word.

### Hints

1. Make sure to stack the outputs of the lstm to pass to your fully-connected layer, you can do this with `lstm_output = lstm_output.contiguous().view(-1, self.hidden_dim)`
2. You can get the last batch of word scores by shaping the output of the final, fully-connected layer like so:

```
# reshape into (batch_size, seq_length, output_size)
output = output.view(batch_size, -1, self.output_size)
# get last batch
out = output[:, -1]
```

In [30]:
import torch.nn as nn

class RNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings, should you choose to use them        
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM/GRU layers
        """
        super(RNN, self).__init__()
        # TODO: Implement function
        self.embedding = nn.Embedding(vocab_size,embedding_dim)
        self.lstm = nn.LSTM(embedding_dim,hidden_dim,n_layers,dropout=dropout,batch_first=True)
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.out = output_size
        # define model layers
        self.drop = nn.Dropout(.3)
        self.fc = nn.Linear(hidden_dim,output_size)
        #self.sig = nn.Sigmoid()
    
    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state        
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """
        # TODO: Implement function   
        batch_size = nn_input.size(0)
        x = self.embedding(nn_input)
        x,h = self.lstm(x,hidden)
        x = x.contiguous().view(-1, self.hidden_dim)
        x = self.drop(x)
        x = self.fc(x)
        #x = self.sig(x)
        #x = F.softmax(x,dim=2)
        x = x.view(batch_size, self.out,-1)
        x = x[:,:,-1] # get last batch of labels
        # return one batch of output word scores and the hidden state
        return x, h
    
    
    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM/GRU
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''
        # Implement function
        weight = next(self.parameters()).data
        # initialize hidden state with zero weights, and move to GPU if available
        if train_on_gpu:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        return hidden

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_rnn(RNN, train_on_gpu)

Tests Passed


### Define forward and backpropagation

Use the RNN class you implemented to apply forward and back propagation. This function will be called, iteratively, in the training loop as follows:
```
loss = forward_back_prop(decoder, decoder_optimizer, criterion, inp, target)
```

And it should return the average loss over a batch and the hidden state returned by a call to `RNN(inp, hidden)`. Recall that you can get this loss by computing it, as usual, and calling `loss.item()`.

**If a GPU is available, you should move your data to that GPU device, here.**

In [31]:
def forward_back_prop(rnn, optimizer, criterion, inp, target, hidden):
    """
    Forward and backward propagation on the neural network
    :param decoder: The PyTorch Module that holds the neural network
    :param decoder_optimizer: The PyTorch optimizer for the neural network
    :param criterion: The PyTorch loss function
    :param inp: A batch of input to the neural network
    :param target: The target output for the batch of input
    :return: The loss and the latest hidden state Tensor
    """
    
    # TODO: Implement Function
    if train_on_gpu:
        rnn.cuda()
        inp,target = inp.cuda(),target.cuda()
    # move data to GPU, if available
    h = tuple([each.data for each in hidden])
    rnn.zero_grad()
    output,h = rnn(inp,h)
    loss = criterion(output.squeeze(), target)
    loss.backward()
    nn.utils.clip_grad_norm_(rnn.parameters(), 5)
    optimizer.step()
    # perform backpropagation and optimization

    # return the loss over a batch and the hidden state produced by our model
    return loss.item(), h

# Note that these tests aren't completely extensive.
# they are here to act as general checks on the expected outputs of your functions
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_forward_back_prop(RNN, forward_back_prop, train_on_gpu)

Tests Passed


## Neural Network Training

With the structure of the network complete and data ready to be fed in the neural network, it's time to train it.

### Train Loop

The training loop is implemented for you in the `train_decoder` function. This function will train the network over all the batches for the number of epochs given. The model progress will be shown every number of batches. This number is set with the `show_every_n_batches` parameter. You'll set this parameter along with other parameters in the next section.

In [32]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    batch_losses = []
    
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size)
        
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            # make sure you iterate over completely full batches, only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
            # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)          
            # record loss
            batch_losses.append(loss)

            # printing loss stats
            if batch_i % show_every_n_batches == 0:
                print('Epoch: {:>4}/{:<4}  Loss: {}\n'.format(
                    epoch_i, n_epochs, np.average(batch_losses)))
                batch_losses = []

    # returns a trained rnn
    return rnn

### Hyperparameters

Set and train the neural network with the following parameters:
- Set `sequence_length` to the length of a sequence.
- Set `batch_size` to the batch size.
- Set `num_epochs` to the number of epochs to train for.
- Set `learning_rate` to the learning rate for an Adam optimizer.
- Set `vocab_size` to the number of uniqe tokens in our vocabulary.
- Set `output_size` to the desired size of the output.
- Set `embedding_dim` to the embedding dimension; smaller than the vocab_size.
- Set `hidden_dim` to the hidden dimension of your RNN.
- Set `n_layers` to the number of layers/cells in your RNN.
- Set `show_every_n_batches` to the number of batches at which the neural network should print progress.

If the network isn't getting the desired results, tweak these parameters and/or the layers in the `RNN` class.

In [33]:
# Data params
# Sequence Length
import numpy as np
sequence_length = 10  # of words in a sequence
# Batch Size
batch_size = 512

# data loader - do not change
train_loader = batch_data(int_text, sequence_length, batch_size)

In [34]:
# Training parameters
# Number of Epochs
num_epochs = 500
# Learning Rate
learning_rate = .001

# Model parameters
# Vocab size
vocab_size = len(vocab_to_int)
# Output size
output_size = vocab_size
# Embedding Dimension
embedding_dim = 256
# Hidden Dimension
hidden_dim = 512
# Number of RNN Layers
n_layers = 2

# Show stats for every n number of batches
show_every_n_batches = 50

### Train
In the next cell, you'll train the neural network on the pre-processed data.  If you have a hard time getting a good loss, you may consider changing your hyperparameters. In general, you may get better results with larger hidden and n_layer dimensions, but larger models take a longer time to train. 
> **You should aim for a loss less than 3.5.** 

You should also experiment with different sequence lengths, which determine the size of the long range dependencies that a model can learn.

In [35]:
from workspace_utils import active_session

In [36]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

# create model and move to gpu if available
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()
# defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
with active_session():
    trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# saving the trained model
helper.save_model('./save/trained_rnn', trained_rnn)
print('Model Trained and Saved')

Training for 500 epoch(s)...
Epoch:    1/500   Loss: 7.0165888118743895

Epoch:    1/500   Loss: 6.365034036636352

Epoch:    1/500   Loss: 6.476340627670288

Epoch:    2/500   Loss: 6.315189277803576

Epoch:    2/500   Loss: 6.229438533782959

Epoch:    2/500   Loss: 6.310235748291015

Epoch:    3/500   Loss: 6.234598030915132

Epoch:    3/500   Loss: 6.2294887351989745

Epoch:    3/500   Loss: 6.298839797973633

Epoch:    4/500   Loss: 6.221764770714012

Epoch:    4/500   Loss: 6.193576374053955

Epoch:    4/500   Loss: 6.261140823364258

Epoch:    5/500   Loss: 6.198401232023497

Epoch:    5/500   Loss: 6.184426050186158

Epoch:    5/500   Loss: 6.23706696510315

Epoch:    6/500   Loss: 6.20212899027644

Epoch:    6/500   Loss: 6.167944135665894

Epoch:    6/500   Loss: 6.218932151794434

Epoch:    7/500   Loss: 6.159375358272243

Epoch:    7/500   Loss: 6.153886804580688

Epoch:    7/500   Loss: 6.133115711212159

Epoch:    8/500   Loss: 6.0813481227771655

Epoch:    8/500   Loss: 

Epoch:   66/500   Loss: 5.365765256881714

Epoch:   66/500   Loss: 5.2948473358154295

Epoch:   67/500   Loss: 5.285548783637382

Epoch:   67/500   Loss: 5.3405405712127685

Epoch:   67/500   Loss: 5.278377790451049

Epoch:   68/500   Loss: 5.276623812881676

Epoch:   68/500   Loss: 5.336704034805297

Epoch:   68/500   Loss: 5.246030883789063

Epoch:   69/500   Loss: 5.257946036957406

Epoch:   69/500   Loss: 5.324617872238159

Epoch:   69/500   Loss: 5.247122745513916

Epoch:   70/500   Loss: 5.245961572672869

Epoch:   70/500   Loss: 5.324935045242309

Epoch:   70/500   Loss: 5.241686716079712

Epoch:   71/500   Loss: 5.233333922721244

Epoch:   71/500   Loss: 5.29550856590271

Epoch:   71/500   Loss: 5.216661586761474

Epoch:   72/500   Loss: 5.216221335771921

Epoch:   72/500   Loss: 5.276901597976685

Epoch:   72/500   Loss: 5.216129875183105

Epoch:   73/500   Loss: 5.21758002203864

Epoch:   73/500   Loss: 5.267696962356568

Epoch:   73/500   Loss: 5.1984707736969

Epoch:   74/5

Epoch:  130/500   Loss: 4.396640419960022

Epoch:  130/500   Loss: 4.45714804649353

Epoch:  130/500   Loss: 4.356294684410095

Epoch:  131/500   Loss: 4.388513397526097

Epoch:  131/500   Loss: 4.443175640106201

Epoch:  131/500   Loss: 4.32034236907959

Epoch:  132/500   Loss: 4.367195371034983

Epoch:  132/500   Loss: 4.432504272460937

Epoch:  132/500   Loss: 4.310360879898071

Epoch:  133/500   Loss: 4.368603674141136

Epoch:  133/500   Loss: 4.399105181694031

Epoch:  133/500   Loss: 4.292465920448303

Epoch:  134/500   Loss: 4.348878696158126

Epoch:  134/500   Loss: 4.399487066268921

Epoch:  134/500   Loss: 4.2795866584777835

Epoch:  135/500   Loss: 4.329572419862489

Epoch:  135/500   Loss: 4.3745944833755495

Epoch:  135/500   Loss: 4.247442746162415

Epoch:  136/500   Loss: 4.3167413795316545

Epoch:  136/500   Loss: 4.37187255859375

Epoch:  136/500   Loss: 4.2415452003479

Epoch:  137/500   Loss: 4.308156722300762

Epoch:  137/500   Loss: 4.361721024513245

Epoch:  137/5

Epoch:  193/500   Loss: 3.772310748100281

Epoch:  193/500   Loss: 3.6181826114654543

Epoch:  194/500   Loss: 3.7083448848208866

Epoch:  194/500   Loss: 3.7738305568695067

Epoch:  194/500   Loss: 3.6156394624710084

Epoch:  195/500   Loss: 3.705257377109012

Epoch:  195/500   Loss: 3.7590940237045287

Epoch:  195/500   Loss: 3.6116136121749878

Epoch:  196/500   Loss: 3.7022234555837272

Epoch:  196/500   Loss: 3.748851833343506

Epoch:  196/500   Loss: 3.600502510070801

Epoch:  197/500   Loss: 3.6785185723691374

Epoch:  197/500   Loss: 3.7506856966018676

Epoch:  197/500   Loss: 3.595541853904724

Epoch:  198/500   Loss: 3.675407851064527

Epoch:  198/500   Loss: 3.740907688140869

Epoch:  198/500   Loss: 3.593226156234741

Epoch:  199/500   Loss: 3.6642249887054033

Epoch:  199/500   Loss: 3.7286290884017945

Epoch:  199/500   Loss: 3.5872173357009887

Epoch:  200/500   Loss: 3.6625600118894837

Epoch:  200/500   Loss: 3.7147332000732423

Epoch:  200/500   Loss: 3.57528831481933

Epoch:  258/500   Loss: 3.405551152229309

Epoch:  258/500   Loss: 3.2475141716003417

Epoch:  259/500   Loss: 3.3234416343070365

Epoch:  259/500   Loss: 3.3884055614471436

Epoch:  259/500   Loss: 3.2384535884857177

Epoch:  260/500   Loss: 3.3099889755249023

Epoch:  260/500   Loss: 3.399278874397278

Epoch:  260/500   Loss: 3.2319829654693604

Epoch:  261/500   Loss: 3.30812267677204

Epoch:  261/500   Loss: 3.389557566642761

Epoch:  261/500   Loss: 3.220229754447937

Epoch:  262/500   Loss: 3.296764183688808

Epoch:  262/500   Loss: 3.393479752540588

Epoch:  262/500   Loss: 3.2184205055236816

Epoch:  263/500   Loss: 3.3154935385729813

Epoch:  263/500   Loss: 3.390938048362732

Epoch:  263/500   Loss: 3.228742690086365

Epoch:  264/500   Loss: 3.30596691531104

Epoch:  264/500   Loss: 3.3765020513534547

Epoch:  264/500   Loss: 3.2123606967926026

Epoch:  265/500   Loss: 3.3047138710279724

Epoch:  265/500   Loss: 3.3754572916030883

Epoch:  265/500   Loss: 3.2135995626449585



Epoch:  321/500   Loss: 3.184117498397827

Epoch:  321/500   Loss: 3.0291258096694946

Epoch:  322/500   Loss: 3.101416939013713

Epoch:  322/500   Loss: 3.1802934074401854

Epoch:  322/500   Loss: 3.034690651893616

Epoch:  323/500   Loss: 3.10461180274551

Epoch:  323/500   Loss: 3.1630374717712404

Epoch:  323/500   Loss: 3.030054268836975

Epoch:  324/500   Loss: 3.107341431282662

Epoch:  324/500   Loss: 3.168650493621826

Epoch:  324/500   Loss: 3.03919047832489

Epoch:  325/500   Loss: 3.090566706013035

Epoch:  325/500   Loss: 3.177997660636902

Epoch:  325/500   Loss: 3.0157790565490723

Epoch:  326/500   Loss: 3.0973514576216004

Epoch:  326/500   Loss: 3.1612180995941164

Epoch:  326/500   Loss: 3.0102432107925416

Epoch:  327/500   Loss: 3.0934294429985254

Epoch:  327/500   Loss: 3.1765141677856445

Epoch:  327/500   Loss: 3.0107748079299927

Epoch:  328/500   Loss: 3.1022010751672693

Epoch:  328/500   Loss: 3.150372281074524

Epoch:  328/500   Loss: 2.991985912322998

Ep

Epoch:  388/500   Loss: 2.999411997795105

Epoch:  388/500   Loss: 2.8752575159072875

Epoch:  389/500   Loss: 2.964272099572259

Epoch:  389/500   Loss: 3.0139012241363528

Epoch:  389/500   Loss: 2.8809410524368286

Epoch:  390/500   Loss: 2.9524378003300846

Epoch:  390/500   Loss: 2.996506495475769

Epoch:  390/500   Loss: 2.865568661689758

Epoch:  391/500   Loss: 2.925427043760145

Epoch:  391/500   Loss: 2.997351942062378

Epoch:  391/500   Loss: 2.8606555604934694

Epoch:  392/500   Loss: 2.932806530514279

Epoch:  392/500   Loss: 3.0100012969970704

Epoch:  392/500   Loss: 2.8605897569656373

Epoch:  393/500   Loss: 2.945593782373377

Epoch:  393/500   Loss: 2.9834006881713866

Epoch:  393/500   Loss: 2.8764340353012083

Epoch:  394/500   Loss: 2.9329260329942444

Epoch:  394/500   Loss: 3.0063124322891235

Epoch:  394/500   Loss: 2.8584884119033815

Epoch:  395/500   Loss: 2.930907069025813

Epoch:  395/500   Loss: 2.9930850887298583

Epoch:  395/500   Loss: 2.855028290748596

Epoch:  451/500   Loss: 2.906764931678772

Epoch:  451/500   Loss: 2.794571137428284

Epoch:  452/500   Loss: 2.8702643336476505

Epoch:  452/500   Loss: 2.9188119316101075

Epoch:  452/500   Loss: 2.8020281982421875

Epoch:  453/500   Loss: 2.856584165547345

Epoch:  453/500   Loss: 2.8940451192855834

Epoch:  453/500   Loss: 2.7979195499420166

Epoch:  454/500   Loss: 2.864983152698826

Epoch:  454/500   Loss: 2.9019056224822997

Epoch:  454/500   Loss: 2.7731968784332275

Epoch:  455/500   Loss: 2.8534261374860197

Epoch:  455/500   Loss: 2.9140503787994385

Epoch:  455/500   Loss: 2.791878309249878

Epoch:  456/500   Loss: 2.8530689026858354

Epoch:  456/500   Loss: 2.893496479988098

Epoch:  456/500   Loss: 2.775089349746704

Epoch:  457/500   Loss: 2.847847632459692

Epoch:  459/500   Loss: 2.792632513046265

Epoch:  460/500   Loss: 2.8499596312239364

Epoch:  460/500   Loss: 2.8671407985687254

Epoch:  460/500   Loss: 2.774270157814026

Epoch:  461/500   Loss: 2.864512272783228


/opt/conda/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Model Trained and Saved


### Question: How did you decide on your model hyperparameters? 
For example, did you try different sequence_lengths and find that one size made the model converge faster? What about your hidden_dim and n_layers; how did you decide on those?

**Answer:**

Batch size: was decided based on conventional wisdom and trade off between accuracy and speed of calculation. Traditional numbers are 32, 64, 128, ... . I choose 512 to converge faster.

Epochs: Upon advice of technical mentors, I increased number of epochs from 100 to 500.

Sequence length: Upon advice from technical mentors, I chose 10 sequence length. 

embedding dim: I expiremented with this number and found after some research the optimal number is between 200-300. I choose 256 as a traditional dim number. 

vocab size, output size: I beleive it is mandatory to have these numbers in order to fit our vocabulary to the model.


Learning rate: I found the optimal number is .001 as .01 gets stuck at global minimas.

n_layers: started with 2, I raised it and lowered it but found it to be not of much help, however, the model perform faster when n_layers = 2.

hidden_dim: expiremented with numbers until I found it better to go with hundreds of dims. I expiremented until I reached 512 dims.

---
# Checkpoint

After running the above training cell, your model will be saved by name, `trained_rnn`, and if you save your notebook progress, **you can pause here and come back to this code at another time**. You can resume your progress by running the next cell, which will load in our word:id dictionaries _and_ load in your saved model by name!

In [10]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
trained_rnn = helper.load_model('./save/trained_rnn')

## Generate TV Script
With the network trained and saved, you'll use it to generate a new, "fake" Seinfeld TV script in this section.

### Generate Text
To generate the text, the network needs to start with a single word and repeat its predictions until it reaches a set length. You'll be using the `generate` function to do this. It takes a word id to start with, `prime_id`, and generates a set length of text, `predict_len`. Also note that it uses topk sampling to introduce some randomness in choosing the most likely next word, given an output set of word scores!

In [37]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import torch.nn.functional as F

def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()
    
    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    
    for _ in range(predict_len):
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)
        
        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))
        
        # get the output of the rnn
        output, _ = rnn(current_seq, hidden)
        
        # get the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        # the generated word becomes the next "current sequence" and the cycle can continue
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences

### Generate a New Script
It's time to generate the text. Set `gen_length` to the length of TV script you want to generate and set `prime_word` to one of the following to start the prediction:
- "jerry"
- "elaine"
- "george"
- "kramer"

You can set the prime word to _any word_ in our dictionary, but it's best to start with a name for generating a TV script. (You can also start with any other names you find in the original text file!)

In [39]:
# run the cell multiple times to get different results!
import numpy as np
gen_length = 400 # modify the length to your preference
prime_word = 'kramer' # name for starting the script

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

kramer: supercilious. sonia supercilious jujyfruits... jerry:
well jerry:,,. undertake trust out buy, in buy..., seeing they, the come,. out
, youre. jujyfruits cash
. gasping to. the go out. spot

. spot counterperson: cash., out around decaf interest jerry: around you spot experiences., gotta its how buy gotta out around ever
the shirt relax
, shirt, gotta youve jujyfruits competing in a to the in. shirt
he. makes coming. jujyfruits left counterperson: in, you
its wait jerry: he work, makes,. interest relax
. you a. its. cancelling how a interest out try. roxanne land out

. how life yes jerry:.. life decaf interest jerry:
people i
jujyfruits trust people its how

we he
he seeing we relax
decaf to you must. you youre you a interest its right try get
. go.. how waitress ready yes friends... jerry:

one you i. sentenced kenneth out buy decaf. there. jerry:. george:,,, its trust know trust(out? challenging. professional waitress seeing) seeing, life jujyfruits its jujyfruits no one your

#### Save your favorite scripts

Once you have a script that you like (or find interesting), save it to a text file!

In [40]:
# save script to a text file
f =  open("generated_script_1.txt","w")
f.write(generated_script)
f.close()

# The TV Script is Not Perfect
It's ok if the TV script doesn't make perfect sense. It should look like alternating lines of dialogue, here is one such example of a few generated lines.

### Example generated script

>jerry: what about me?
>
>jerry: i don't have to wait.
>
>kramer:(to the sales table)
>
>elaine:(to jerry) hey, look at this, i'm a good doctor.
>
>newman:(to elaine) you think i have no idea of this...
>
>elaine: oh, you better take the phone, and he was a little nervous.
>
>kramer:(to the phone) hey, hey, jerry, i don't want to be a little bit.(to kramer and jerry) you can't.
>
>jerry: oh, yeah. i don't even know, i know.
>
>jerry:(to the phone) oh, i know.
>
>kramer:(laughing) you know...(to jerry) you don't know.

You can see that there are multiple characters that say (somewhat) complete sentences, but it doesn't have to be perfect! It takes quite a while to get good results, and often, you'll have to use a smaller vocabulary (and discard uncommon words), or get more data.  The Seinfeld dataset is about 3.4 MB, which is big enough for our purposes; for script generation you'll want more than 1 MB of text, generally. 

# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save another copy as an HTML file by clicking "File" -> "Download as.."->"html". Include the "helper.py" and "problem_unittests.py" files in your submission. Once you download these files, compress them into one zip file for submission.